In [0]:
# Copyright 2018 The TensorFlow GAN Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

TensorFlow 2.x selected.


In [0]:
# Setting up
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # tf_version only exist in colab
  %tensorflow_version 2.x

except Exception:
  pass

# SETUP

In [0]:
import tensorflow as tf
tf.random.set_seed(7)
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
import os
import time

# HYPERPARAMETERS

X_Train is the array of images, the user wants to use as training data. Labels and test data aren't needed

In [0]:
EPOCHS = 60
BATCH_SIZE = 256
NO_OF_BATCHES = int(X_Train.shape[0]/BATCH_SIZE)
HALF_BATCH_SIZE = 128
NOISE_DIM = 100
# Customised ADAM optimizer
adam = tf.keras.optimizers.Adam(lr=2e-4,beta_1=0.5) # lr: Learning Rate

# Making up models for GAN

In [0]:
from tensorflow.keras.layers import *
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model

# GENERATOR 

In [0]:
# Generator
generator = Sequential()
generator.add(Dense(256,input_shape=(NOISE_DIM,)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784,activation='tanh'))

# Compile
generator.compile(loss='binary_crossentropy',optimizer=adam)

generator.summary()

# DISCRIMINATOR

In [0]:
#Discriminator
discriminator = Sequential()
discriminator.add(Dense(512,input_shape=(784,)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1,activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy',optimizer=adam)
discriminator.summary()

# Combined Model

In [0]:
# Combined Model (Geneerator + Discriminator) -> Functional API
discriminator.trainable = False
gan_input = Input(shape=(NOISE_DIM,))
generator_output = generator(gan_input)
gan_output = discriminator(generator_output)

model = Model(gan_input,gan_output)
model.compile(loss='binary_crossentropy',optimizer=adam)

model.summary()

In [0]:
def showImgs(epoch):
    noise = np.random.normal(0,1,size=(100,NOISE_DIM)) # Random Noise
    generated_imgs = generator.predict(noise)
    generated_imgs = generated_imgs.reshape(-1,28,28)
    
    #Display the Images
    plt.figure(figsize=(10,10))
    for i in range(100):
        plt.subplot(10,10,i+1)
        plt.imshow(generated_imgs[i],cmap='gray',interpolation='nearest')
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Training

In [0]:
d_losses = []
g_losses =  []

# Training Loop
for epoch in range(EPOCHS+1): 
    epoch_d_loss = 0.0
    epoch_g_loss = 0.0
    
    # Mini Batch
    for step in range(NO_OF_BATCHES):
        idx = np.random.randint(0,X_Train.shape[0],HALF_BATCH_SIZE)
        real_imgs = X_Train[idx]
        
        # generate fake images assuming generator is frozen
        noise = np.random.normal(0,1,size=(HALF_BATCH_SIZE,NOISE_DIM))
        fake_imgs = generator.predict(noise)
        
        # Labels
        real_y = np.ones((HALF_BATCH_SIZE,1))*0.9
        fake_y = np.zeros((HALF_BATCH_SIZE,1))
        
        #Train on Real and Fake Images
        d_real_loss = discriminator.train_on_batch(real_imgs,real_y)
        d_fake_loss = discriminator.train_on_batch(fake_imgs,fake_y)
        
        d_loss = 0.5*d_real_loss + 0.5*d_fake_loss
        epoch_d_loss += d_loss
        
        # Train Generator
        noise = np.random.normal(0,1,size=(BATCH_SIZE,NOISE_DIM))
        real_y = np.ones((BATCH_SIZE,1))
        g_loss = model.train_on_batch(noise,real_y)
        epoch_g_loss += g_loss
        
    d_losses.append(epoch_d_loss)
    g_losses.append(epoch_g_loss)
        
    print("Epoch %d D Loss %.4f G loss %0.4f "%((epoch+1),epoch_d_loss,epoch_g_loss))
    if (epoch%5)==0: #printing images after every 5 epocs
        #generator.save("model/gen_{0}.h5".format(epoch)) # Saving the model
        showImgs(epoch) # Show the image transformation progress at every step